In [ ]:
!curl http://imagenet.stanford.edu/internal/car196/cars_test.tgz --output /content/test_images.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  932M  100  932M    0     0  39.9M      0  0:00:23  0:00:23 --:--:-- 41.4M


In [ ]:
!curl http://imagenet.stanford.edu/internal/car196/cars_train.tgz --output /content/train_images.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  933M  100  933M    0     0  45.7M      0  0:00:20  0:00:20 --:--:-- 47.4M


In [ ]:
!mkdir /content/images
!tar -xvzf /content/test_images.tgz -C /content/images/

Streaming output truncated to the last 5000 lines.
cars_test/03042.jpg
cars_test/03043.jpg
cars_test/03044.jpg
cars_test/03045.jpg
cars_test/03046.jpg
cars_test/03047.jpg
cars_test/03048.jpg
cars_test/03049.jpg
cars_test/03050.jpg
cars_test/03051.jpg
cars_test/03052.jpg
cars_test/03053.jpg
cars_test/03054.jpg
cars_test/03055.jpg
cars_test/03056.jpg
cars_test/03057.jpg
cars_test/03058.jpg
cars_test/03059.jpg
cars_test/03060.jpg
cars_test/03061.jpg
cars_test/03062.jpg
cars_test/03063.jpg
cars_test/03064.jpg
cars_test/03065.jpg
cars_test/03066.jpg
cars_test/03067.jpg
cars_test/03068.jpg
cars_test/03069.jpg
cars_test/03070.jpg
cars_test/03071.jpg
cars_test/03072.jpg
cars_test/03073.jpg
cars_test/03074.jpg
cars_test/03075.jpg
cars_test/03076.jpg
cars_test/03077.jpg
cars_test/03078.jpg
cars_test/03079.jpg
cars_test/03080.jpg
cars_test/03081.jpg
cars_test/03082.jpg
cars_test/03083.jpg
cars_test/03084.jpg
cars_test/03085.jpg
cars_test/03086.jpg
cars_test/03087.jpg
cars_test/03088.jpg
cars_test

In [ ]:
!tar -xvzf /content/train_images.tgz -C /content/images/

Streaming output truncated to the last 5000 lines.
cars_train/03145.jpg
cars_train/03146.jpg
cars_train/03147.jpg
cars_train/03148.jpg
cars_train/03149.jpg
cars_train/03150.jpg
cars_train/03151.jpg
cars_train/03152.jpg
cars_train/03153.jpg
cars_train/03154.jpg
cars_train/03155.jpg
cars_train/03156.jpg
cars_train/03157.jpg
cars_train/03158.jpg
cars_train/03159.jpg
cars_train/03160.jpg
cars_train/03161.jpg
cars_train/03162.jpg
cars_train/03163.jpg
cars_train/03164.jpg
cars_train/03165.jpg
cars_train/03166.jpg
cars_train/03167.jpg
cars_train/03168.jpg
cars_train/03169.jpg
cars_train/03170.jpg
cars_train/03171.jpg
cars_train/03172.jpg
cars_train/03173.jpg
cars_train/03174.jpg
cars_train/03175.jpg
cars_train/03176.jpg
cars_train/03177.jpg
cars_train/03178.jpg
cars_train/03179.jpg
cars_train/03180.jpg
cars_train/03181.jpg
cars_train/03182.jpg
cars_train/03183.jpg
cars_train/03184.jpg
cars_train/03185.jpg
cars_train/03186.jpg
cars_train/03187.jpg
cars_train/03188.jpg
cars_train/03189.jpg
cars

In [ ]:
!curl https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz --output /content/labels.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  323k  100  323k    0     0   538k      0 --:--:-- --:--:-- --:--:--  537k


In [ ]:
!tar -xvzf /content/labels.tgz

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [ ]:
# !cat /content/devkit/README.txt

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy.io as sio
import os

batch_size = 32
img_size = 224
data_dir = '/content/images/'

In [ ]:
train_annots = sio.loadmat('/content/devkit/cars_train_annos.mat')
test_annots = sio.loadmat('/content/devkit/cars_test_annos.mat')
classes = sio.loadmat('/content/devkit/cars_meta.mat')['class_names'][0]

In [ ]:
train_labels = train_annots['annotations']['class'] #starts from 1 index
train_labels.shape

(1, 8144)

In [ ]:
train_labels = train_labels.reshape((8144)).astype('uint8') #reshaped to tf compatible shape
train_labels = train_labels - 1 # converted to 0 index
train_labels = tf.one_hot(train_labels, len(classes))
train_labels.shape

TensorShape([8144, 196])

In [ ]:
from PIL.Image import open
import glob
import natsort
train_img_paths = glob.glob(data_dir + 'cars_train/*.jpg')
test_img_paths = glob.glob(data_dir + 'cars_test/*.jpg')
train_img_paths = natsort.natsorted(train_img_paths,reverse=False)
test_img_paths = natsort.natsorted(test_img_paths,reverse=False)
def load_image(path):
    # path/label represent values for a single example
    image = open(path)
    image = tf.convert_to_tensor(np.array(image))
    if len(image.shape) == 2:
      image = image[:, :, None]
    else:
      image = tf.image.rgb_to_grayscale(image)
    # print(image.shape)
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.image.per_image_standardization(image)
    return image
train_images = np.empty((len(train_img_paths), img_size, img_size, 1), dtype='float32')
test_images = np.empty((len(test_img_paths), img_size, img_size, 1), dtype='float32')
for i in range(len(train_img_paths)):
  train_images[i] = load_image(train_img_paths[i])
  print(f"{i}/{len(train_img_paths)}")
for i in range(len(test_img_paths)):
  train_images[i] = load_image(test_img_paths[i])
  print(f"{i}/{len(test_img_paths)}")

Streaming output truncated to the last 5000 lines.
3041/8041
3042/8041
3043/8041
3044/8041
3045/8041
3046/8041
3047/8041
3048/8041
3049/8041
3050/8041
3051/8041
3052/8041
3053/8041
3054/8041
3055/8041
3056/8041
3057/8041
3058/8041
3059/8041
3060/8041
3061/8041
3062/8041
3063/8041
3064/8041
3065/8041
3066/8041
3067/8041
3068/8041
3069/8041
3070/8041
3071/8041
3072/8041
3073/8041
3074/8041
3075/8041
3076/8041
3077/8041
3078/8041
3079/8041
3080/8041
3081/8041
3082/8041
3083/8041
3084/8041
3085/8041
3086/8041
3087/8041
3088/8041
3089/8041
3090/8041
3091/8041
3092/8041
3093/8041
3094/8041
3095/8041
3096/8041
3097/8041
3098/8041
3099/8041
3100/8041
3101/8041
3102/8041
3103/8041
3104/8041
3105/8041
3106/8041
3107/8041
3108/8041
3109/8041
3110/8041
3111/8041
3112/8041
3113/8041
3114/8041
3115/8041
3116/8041
3117/8041
3118/8041
3119/8041
3120/8041
3121/8041
3122/8041
3123/8041
3124/8041
3125/8041
3126/8041
3127/8041
3128/8041
3129/8041
3130/8041
3131/8041
3132/8041
3133/8041
3134/8041
3135/8041

In [ ]:
input = tf.keras.layers.Input((img_size, img_size, 1))
model = tf.keras.applications.VGG16(
    include_top=True, weights=None, input_tensor=input,
    input_shape=None, pooling=None, classes=len(classes),
    classifier_activation='softmax'
)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=100)

255/255 [==============================] - 109s 426ms/step - loss: 5.2789 - accuracy: 0.0070


In [ ]:
history.history

{'accuracy': [0.006999017670750618], 'loss': [5.2788543701171875]}